# Import Library

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os 
path = os.getcwd()
workdir=os.path.join(path,'drive','My Drive','Colab Notebooks')
print(workdir)
os.chdir(workdir)

/content/drive/My Drive/Colab Notebooks


In [3]:
!pip install pyradiomics

In [4]:
from pathlib import Path
import numpy as np
from PIL import Image
import pandas as pd
import os
import h5py
from tqdm.notebook import tqdm
from radiomics import featureextractor
import SimpleITK as sitk
import pprint
from scipy import ndimage
import logging
import pprint
# limit gpu memory
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [0]:
def create_dir(path):
    dir_path = Path(path)
    os.makedirs(dir_path, exist_ok = True)
    return dir_path

data_dir = Path('./')
img_dir = create_dir('./new_img')
mask_dir = create_dir('./new_mask')
#processed_img_dir = create_dir('./processed_img')
#feature_dir = create_dir('./data/feature')

# Load Label

In [7]:
def map_label(label):
    if label == 'normal':
        return 0
    elif label == 'pneumonia':
        return 1
    elif label == 'COVID-19':
        return 2
    else:
        return None

label_path = Path('./data/train_split_v2.txt')
metadata_path = Path('./data/metadata.csv')
df = pd.read_csv(label_path, names=['aux', 'file_name', 'label'], sep=' ')
df = df.set_index('file_name')
df = df.drop(columns=['aux'])
df = df.loc[~df.index.duplicated(keep='first')]
df.head()
#df = pd.read_csv(metadata_path, index_col=0)

label = df['label'].values
int_label = [map_label(x) for x in label]
int_label = np.array(int_label)
df['int_label'] = int_label
df['name'] = [os.path.splitext(x)[0] for x in df.index.values]
df.to_csv(metadata_path, ',')
meta_df = df.copy()
meta_df = meta_df.set_index('name')
df.head()

,label,int_label,name
file_name,,,
SARS-10.1148rg.242035193-g04mr34g0-Fig8a-day0.jpeg,pneumonia,1,SARS-10.1148rg.242035193-g04mr34g0-Fig8a-day0
SARS-10.1148rg.242035193-g04mr34g0-Fig8b-day5.jpeg,pneumonia,1,SARS-10.1148rg.242035193-g04mr34g0-Fig8b-day5
SARS-10.1148rg.242035193-g04mr34g0-Fig8c-day10.jpeg,pneumonia,1,SARS-10.1148rg.242035193-g04mr34g0-Fig8c-day10
SARS-10.1148rg.242035193-g04mr34g04a-Fig4a-day7.jpeg,pneumonia,1,SARS-10.1148rg.242035193-g04mr34g04a-Fig4a-day7
SARS-10.1148rg.242035193-g04mr34g04b-Fig4b-day12.jpeg,pneumonia,1,SARS-10.1148rg.242035193-g04mr34g04b-Fig4b-day12


In [8]:
print(np.unique(df['label'].values))
print(len(df['label'].values))

['COVID-19' 'normal' 'pneumonia']
13494


# Generate mask

In [6]:
#imgs = df.index.values
imgs = os.listdir('new_img')
from skimage import transform, io, img_as_float, exposure

img_arr = []
original_size = []
for i in tqdm(range(len(imgs))):
    ## get img in array of [0..1]
    img = img_as_float(io.imread(img_dir / imgs[i]))
   
    ## some img have 3 channels for some reason
    ## I will just take the first one
    if len(img.shape) > 2:
        img = img[:, :, 0]
    
    original_size.append(img.shape)
    ## resize to 256, 256
    im_shape = (256, 256)
    img = transform.resize(img, im_shape)
    
    ## adjust exposure
    img = exposure.equalize_hist(img)
    
    ## convert (256, 256) to (256, 256, 1)
    img = np.expand_dims(img, -1)
    img_arr.append(img)

In [0]:
## center and standardize img
img_arr = np.array(img_arr)
img_arr -= np.mean(img_arr)
img_arr /= np.std(img_arr)

In [8]:
with h5py.File(data_dir / 'standardized_256x256_data.h5', 'w') as file:
    for i in tqdm(range(img_arr.shape[0])):
        file.create_dataset('img/{:06d}'.format(i), data =img_arr[i])
        file.create_dataset('size/{:06d}'.format(i), data =original_size[i])

In [9]:
img_arr = []
original_size = []
with h5py.File(data_dir / 'standardized_256x256_data.h5', 'r') as file:
    keys = file['img'].keys()
    total = len(keys)
    for i in tqdm(range(total)):
        img_arr.append(file.get('img/{:06d}'.format(i))[:])
        original_size.append(file.get('size/{:06d}'.format(i))[:])
img_arr = np.array(img_arr)
original_size = np.array(original_size)

In [10]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage import morphology, color, io, exposure, transform, img_as_bool

def remove_small_regions(img, size):
    """Morphologically removes small (less than size) connected regions of 0s or 1s."""
    img = morphology.remove_small_objects(img, size)
    img = morphology.remove_small_holes(img, size)
    return img

im_shape = (256, 256)
inp_shape = img_arr[0].shape
model_path = Path('./trained_model.hdf5')
model = load_model(model_path)
test_gen = ImageDataGenerator(rescale=1.)
for img_sample, size, name in tqdm(zip(img_arr, original_size, imgs), total=len(img_arr)):
    # generate mask prediction
    img_sample = np.reshape(img_sample, (1, 256, 256, 1))
    img = exposure.rescale_intensity(np.squeeze(img_sample), out_range=(0,1))
    pred = model.predict(img_sample)[..., 0].reshape(inp_shape[:2])

    # Binarize masks
    pr = pred > 0.5
    
    # Remove regions smaller than 2% of the image
    pr = remove_small_regions(pr, 0.02 * np.prod(im_shape))
   
    # upscale mask to original image
    resized_mask = img_as_bool(transform.resize(pr, size))
    
    # save mask
    im = Image.fromarray(resized_mask)
    im.save(mask_dir / '{}'.format(name))            

# Preprocessing img data

In [0]:
imgs = df.index.values
from skimage import transform, io, img_as_float, exposure

img_arr = []
for i in tqdm(range(len(imgs))):
    ## get img in array of [0..1]
    img = io.imread(img_dir / imgs[i])
    
    ## some img have 3 channels for some reason
    ## I will just take the first one
    if len(img.shape) > 2:
        img = img[:, :, 0]   
    
    ## adjust exposure
    img = exposure.equalize_hist(img)
    
    ## convert (256, 256) to (256, 256, 1)
    img *= 255.
    img = img.astype(np.uint8)
    img = Image.fromarray(img)
    img.save(processed_img_dir / imgs[i])

In [0]:

img_list = df.index.values
img_list.sort()
mask_list = df.index.values
mask_list.sort()

print(img_list[:5])
print(mask_list[:5])

['000924cf-0f8d-42bd-9158-1af53881a557.png', '000fe35a-2649-43d4-b027-e67796d412e0.png', '0010f549-b242-4e94-87a8-57d79de215fc.png', '001916b8-3d30-4935-a5d1-8eaddb1646cd.png', '0022073f-cec8-42ec-ab5f-bc2314649235.png']
['000924cf-0f8d-42bd-9158-1af53881a557.png', '000fe35a-2649-43d4-b027-e67796d412e0.png', '0010f549-b242-4e94-87a8-57d79de215fc.png', '001916b8-3d30-4935-a5d1-8eaddb1646cd.png', '0022073f-cec8-42ec-ab5f-bc2314649235.png']


In [0]:
logger = logging.getLogger('radiomics')
logger.setLevel(logging.CRITICAL)  # Set default level of logger to INFO to reflect most common setting for a log file
extractor = featureextractor.RadiomicsFeatureExtractor()


def delete_entry(fe):
    keys = fe.keys()
    delete_keys = []
    for i in keys:
        if 'diagnostics' in i:
            delete_keys.append(i)
    
    for i in delete_keys:
        del fe[i]
    return fe

def save_feature(features, save_path, label):
    if len(features) == 0:
        return
    
    dfs = []
    for i in range(len(features)):
        df = pd.DataFrame(features[i], index=[i])
        df['label'] = label
        dfs.append(df)
        
    df = pd.concat(dfs)    
    print(save_path, len(features), label)
    df.to_csv(save_path) 
    
# for i in tqdm(range(40, 45)):

for i in tqdm(range(9000, len(img_list))):
    img = np.array(Image.open(processed_img_dir / img_list[i]))
    img = sitk.GetImageFromArray(img)

    mask = np.array(Image.open(mask_dir / mask_list[i]))
    mask = mask.astype('uint8')
    mask, label = ndimage.label(mask)
    
    features = []    
    for j in range(1, label+1):
        temp_mask = mask.copy()
        temp_mask[temp_mask != j] = 0
        temp_mask[temp_mask == j] = 1
    
        check_sum = np.sum(temp_mask)
        if check_sum < 50000:
            continue
        
        temp_mask = sitk.GetImageFromArray(temp_mask)
        feature = extractor.execute(img, temp_mask)
        feature = delete_entry(feature)
        features.append(feature)
               
    save_feature(features, feature_dir / '{}{}'.format(os.path.splitext(img_list[i])[0], '.csv'), meta_df.loc[os.path.splitext(img_list[i])[0]]['int_label'])

data/feature/af8a9a3f-9487-454b-8ee6-65c9554f3a87.csv 2 0
data/feature/af8ac1a0-5f55-42e1-9cf7-d79e70bf5fb1.csv 2 0
data/feature/af8b2dd5-e469-4be9-bdb3-027d7a1a3dc0.csv 2 0
data/feature/af8b563a-6874-45b6-b1fa-8c047177a7aa.csv 2 0
data/feature/af8fc053-b69b-4915-9ea8-f538028f9b2c.csv 2 0
data/feature/af935971-e768-468a-8bbe-cdb025daec83.csv 2 0
data/feature/af9788f6-93e8-4460-aadd-5074c5b3b7fa.csv 2 0
data/feature/af98dfd4-7155-481f-b07e-fbecf6ddc40d.csv 2 0
data/feature/af9d342a-518e-47be-99a3-b2138960db46.csv 2 0
data/feature/afa1bff5-3158-4bd5-bfe4-e85bcd3577d7.csv 2 0
data/feature/afa2fcb2-e578-45c7-97d4-cb56488d80c5.csv 2 0
data/feature/afa5b7eb-938a-4c84-ad90-0ea94ffa6878.csv 2 0
data/feature/afa95e84-6a86-4203-ad3c-f3adb8aaaa03.csv 2 0
data/feature/afaeb13e-137c-415b-aa87-1f329432aa1c.csv 2 0
data/feature/afb20c9c-a4bf-4a02-99ea-50d134dadb6b.csv 2 0
data/feature/afb87feb-0409-42d0-b5cf-7e6a820f67c0.csv 1 0
data/feature/afba92c7-0822-4350-bd6a-43706d515b70.csv 2 0
data/feature/a

In [0]:
csv_names = os.listdir(feature_dir)

labels = []
shape = []
covid_19 =[]
for i in tqdm(range(len(csv_names))):
    df = pd.read_csv(feature_dir / csv_names[i], index_col=0)
    labels.append(df['label'].values[0])
    shape.append(df.shape[0])
    if df['label'].values[0] == 2:
        covid_19.append(df.shape[0])
    
labels = np.array(labels)
shape = np.array(shape)
covid_19 = np.array(covid_19)

In [0]:
from collections import Counter
print(Counter(labels))
print(Counter(shape))
print(Counter(covid_19))
print(Counter(int_label))

Counter({0: 7908, 1: 5384, 2: 87})
Counter({2: 12837, 1: 483, 3: 58, 4: 1})
Counter({2: 85, 3: 1, 1: 1})
Counter({0: 7966, 1: 5439, 2: 89})


# Generate Aggregation CSV

## Average all feature

In [0]:
csv_names = os.listdir(feature_dir)
labels = []
shape = []

df = pd.DataFrame()
for i in tqdm(range(len(csv_names))):
# for i in tqdm(range(10)):

    temp_df = pd.read_csv(feature_dir / csv_names[i], index_col=0)
    temp_dict = {}
#     temp_dict['label'] = temp_df['label'].values[0]
    temp_dict['label'] = meta_df.loc[os.path.splitext(csv_names[i])[0]]['int_label']

    temp_dict['shape'] = temp_df.shape[0]
    temp_dict['name'] = os.path.splitext(csv_names[i])[0]
    
    temp_df = temp_df.drop(columns=['label'])
    temp_cols = temp_df.columns.values
    for col in temp_cols:
        temp_dict[col] = np.mean(temp_df[col].values)
        
    df = df.append(temp_dict, ignore_index=True)
#     display(df.head())
#     break
df.to_csv(Path('./data/average_aggr_feature.csv'), index=False)
    

## Separate left and right lung

In [0]:
csv_names = os.listdir(feature_dir)
labels = []
shape = []
df = pd.DataFrame()

for i in tqdm(range(len(csv_names))):
    temp_df = pd.read_csv(feature_dir / csv_names[i], index_col=0)
    if temp_df.shape[0] != 2:
        continue
    
    temp_dict = {}
#     temp_dict['label'] = temp_df['label'].values[0]
    temp_dict['label'] = meta_df.loc[os.path.splitext(csv_names[i])[0]]['int_label']
    temp_dict['shape'] = temp_df.shape[0]
    temp_dict['name'] = os.path.splitext(csv_names[i])[0]
    
    temp_df = temp_df.drop(columns=['label'])
    temp_cols = temp_df.columns.values
    for col in temp_cols:
        values = temp_df[col].values
        left = values[0]
        right = values[1]
        
        temp_dict['{}_{}'.format('left', col)] = left
        temp_dict['{}_{}'.format('right', col)] = right
        
    df = df.append(temp_dict, ignore_index=True)
#     pprint.pprint(temp_dict)
#     break

df.to_csv(Path('./data/left_right_aggr_feature.csv'), index=False)